In [ ]:
import os
import sys
import shutil
import subprocess
import gc
import xarray as xr
import numpy as np
import pymp

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import rc
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.transforms import Bbox
from matplotlib.patches import FancyBboxPatch
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from scipy.interpolate import interp1d
from scipy.interpolate import interp2d

path = os.getcwd().split('/')
machine_path = '/'+path[1]+'/'+path[2] #cat the /home/user/ or /Users/user from system using path

sys.path.insert(0, f"{machine_path}/opt/mandyoc-scripts/functions")
from mandyocIO import read_mandyoc_output, read_datasets, read_particle_path, single_plot, change_dataset, plot_property, _extract_interface, find_nearest, _read_step

In [ ]:
# Initial setup
path = os.getcwd().split('/') # Get local file
machine_path = '/'+path[1]+'/'+path[2] # Select home according to OS.

# Path to the model directory

#Keel experiments

# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx3000km/cold_keel/keel_stable_DT200_HprodAst_Lx3000km_ck'
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx3000km/cold_keel/keel_stable_DT290_HprodAst_Lx3000km_ck'
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx3000km/cold_keel/keel_stable_DT350_HprodAst_Lx3000km_ck'

# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/keel_stable_DT200_HprodAst_Lx4000km_ck'
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/keel_stable_DT290_HprodAst_Lx4000km_ck'
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/keel_stable_DT350_HprodAst_Lx4000km_ck'

# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/ast_wind/keel_stable_DT200_HprodAst_Lx4000km_ck_aw'
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/ast_wind/keel_stable_DT290_HprodAst_Lx4000km_ck_aw'
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/ast_wind/keel_stable_DT350_HprodAst_Lx4000km_ck_aw'


#Thermal boundary condition fixed

#REFERENCE
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/keel_stable_DT200_HprodAst_Lx4000km_ck_tbcf' #took

#Effect of the asthenospheric wind on reference scenario
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/ast_wind/keel_stable_DT200_HprodAst_Lx4000km_ck_aw_tbcf'

#Shift craton
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/shift_craton/keel_stable_DT200_HprodAst_Lx4000km_ck_tbcf_sc' #took
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/shift_craton/keel_stable_DT200_HprodAst_Lx4000km_ck_aw_tbcf_sc' #asthenospheric wind

#Mobile belt

#deeper mobile belt
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/mobile_belt/mb_dryol/keel_stable_DT200_HprodAst_Lx4000km_ck_mbdo' #took
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/mobile_belt/mb_wetol/keel_stable_DT200_HprodAst_Lx4000km_ck_mbwo' #took

#shallow mobile belt
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/mobile_belt/shallow_mb/mb_dryol/keel_stable_DT200_HprodAst_Lx4000km_ck_shallow_mbdo_Cmb001'
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/mobile_belt/shallow_mb/mb_dryol/keel_stable_DT200_HprodAst_Lx4000km_ck_shallow_mbdo_Cmb01'
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/mobile_belt/shallow_mb/mb_wetol/keel_stable_DT200_HprodAst_Lx4000km_ck_shallow_mbwo_Cmb1'
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/mobile_belt/shallow_mb/mb_wetol/keel_stable_DT200_HprodAst_Lx4000km_ck_shallow_mbwo_Cmb3'

#long craton
scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/long_craton/keel_stable_DT200_HprodAst_Lx4000km_ck_lc'
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/long_craton/mobile_belt/mb_dryol/keel_stable_DT200_HprodAst_Lx4000km_ck_lc_shallow_mbdo_Cmb001'
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/long_craton/mobile_belt/mb_wetol/keel_stable_DT200_HprodAst_Lx4000km_ck_lc_shallow_mbwo_Cmb1'
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/long_craton/mobile_belt/mb_wetol/keel_stable_DT200_HprodAst_Lx4000km_ck_lc_shallow_mbwo_Cmb5'

#Weak keel
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/weak_keel/keel_stable_DT200_PT1292oC_HprodAst_Lx4000km_ck_tbcf_wk_Cmlit001' #took
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/weak_keel/keel_stable_DT200_PT1292oC_HprodAst_Lx4000km_ck_tbcf_wk_Cmlit01' #took
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/weak_keel/keel_stable_DT200_PT1292oC_HprodAst_Lx4000km_ck_tbcf_wk_Cmlit02' #took
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/weak_keel/keel_stable_DT200_PT1292oC_HprodAst_Lx4000km_ck_tbcf_wk_Cmlit025' #took
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/weak_keel/keel_stable_DT200_PT1292oC_HprodAst_Lx4000km_ck_tbcf_wk_Cmlit03' #took
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/weak_keel/keel_stable_DT200_PT1292oC_HprodAst_Lx4000km_ck_tbcf_wk_Cmlit05' #took

#variation of Tp and asthenospheric wind
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/ast_wind/keel_stable_DT350_HprodAst_Lx4000km_ck_aw_tbcf'

#PARTIAL
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/mobile_belt/mb_wetol/keel_stable_DT200_HprodAst_Lx4000km_ck_mbwo/mbwo_partial'
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/mobile_belt/mb_dryol/keel_stable_DT200_HprodAst_Lx4000km_ck_mbdo/mbdo_partial'

# local = True
local = False

if(local==True):
    model_path = machine_path + scenario
else:
    # scenario = '/RFT_Clc1_DT200_PT1292oC_lit80km_1x1km2_NHK'
    external_media = 'Joao_Macedo'
    model_path = f"/media/{machine_path.split('/')[-1]}/{external_media}{scenario}"

dataset = xr.open_dataset(f'{model_path}/_output_temperature.nc')

Nx = int(dataset.nx)
Nz = int(dataset.nz)
Lx = float(dataset.lx)
Lz = float(dataset.lz)

instant = dataset.time[-1]

x = np.linspace(0, Lx/1000.0, Nx)
z = np.linspace(Lz/1000.0, 0, Nz)
xx, zz  = np.meshgrid(x, z)

print(dataset.info)

In [ ]:
scenario_name = scenario.split('/')[-1]
properties = scenario_name.split('_')

# thickness of sticky air layer (m)
thickness_sa = 40 * 1.0e3
# thickness of upper crust (m)
thickness_upper_crust = 20 * 1.0e3
# thickness of lower crust (m)
thickness_lower_crust = 15 * 1.0e3
# total thickness of lithosphere (m)
thickness_litho = 80 * 1.0e3
#shift craton

mobile_belt = False
shift_craton = 0.0e3
Lcraton = 1200.0e3

if(('mbdo' in properties) or ('mbwo' in properties)):
    mobile_belt = True

if('sc' in properties):
    shift_craton = 700.0e3

if('lc' in properties):
    Lcraton = 2000.0e3 #m
        
print(f"Mobile Belt: {mobile_belt}")
print(f"Shift craton: {shift_craton}")
print(f"Length of craton: {Lcraton/1.0e3} km")

#Building the interfaces
if(mobile_belt == False):
        interfaces = {
                "litho": np.ones(Nx) * (thickness_litho + thickness_sa),
                "lower_crust": np.ones(Nx) * (thickness_lower_crust + thickness_upper_crust + thickness_sa),
                "upper_crust": np.ones(Nx) * (thickness_upper_crust + thickness_sa),
                "air": np.ones(Nx) * (thickness_sa),
                }

        dx = Lx/(Nx-1)
        # Lcraton = 600.0e3 #m
        # Lcraton = 1200.0e3 #m
        # Lcraton = 2000.0e3 #m
        thickening = thickness_litho + 120.e3 #m
        Ncraton = int(Lcraton//dx) #largura em indices
        
        Nshift = int(shift_craton//dx)
        interfaces['litho'][Nx//2 - Ncraton//2 + Nshift : Nx//2 + Ncraton//2 + Nshift] = thickness_sa + thickening

else: #with mobile belt
    interfaces = {
        "litho_LAB": np.ones(Nx) * (thickness_litho + thickness_sa), #lab horizontal
        "litho_HETERO": np.ones(Nx) * (thickness_litho + thickness_sa), #interface entre central e lateral -  interface 
        "lower_crust": np.ones(Nx) * (thickness_lower_crust + thickness_upper_crust + thickness_sa),
        "upper_crust": np.ones(Nx) * (thickness_upper_crust + thickness_sa),
        "air": np.ones(Nx) * (thickness_sa),
        }

    #Building craton
    dx = Lx/(Nx-1)
    # Lcraton = 600.0e3 #m
    # Lcraton = 1200.0e3 #m
#     Lcraton = 2000.0e3 #m
    thickening = thickness_litho + 120.e3 #m

    Ncraton = int(Lcraton//dx) #largura em indices

    Nshift = int(shift_craton//dx)

    interfaces['litho_LAB'][Nx//2 - Ncraton//2 + Nshift : Nx//2 + Ncraton//2 + Nshift] = thickness_sa + thickening
    
    #Building mobile belt
    interfaces['litho_HETERO'][Nx//2 - Ncraton//2 + Nshift : Nx//2 + Ncraton//2 + Nshift] = thickness_sa + thickening
    Lmb = 300.0e3 #length of mobile belt
    N_Lmb = int(Lmb//dx)
    # thinning = 50.0e3
    thinning = 100.0e3

    interfaces['litho_HETERO'][Nx//2 - N_Lmb//2 + Nshift : Nx//2 + N_Lmb//2 + Nshift] = thickness_sa + thickening - thinning


In [ ]:
# unzip_steps = False
unzip_steps = True

if(unzip_steps):
    !unzip -o {model_path}/{scenario_name}.zip step*.txt -d {model_path}

In [ ]:
ncores = 20
# i = 0
# i = 100
i = 200

data_x, data_z, data_ID, data_lithology, data_strain = _read_step(model_path, f"step_{int(dataset.step[i].values*1)}_", ncores)

xcenter = (Lx/2) + shift_craton

#selecting particles out of the keel, with z deeper than lithosphere km and lithology = 1 or 2 (lithosphere and lithosphere with mobile belt)
if(mobile_belt):
    cond_out_keel = ((data_x >= 0) & (data_x < xcenter - Lcraton/2) | (data_x > xcenter + Lcraton/2) & (data_x <= Lx))\
                & (data_z < -(thickness_litho + thickness_sa + 10.0e3))\
                & ( (data_lithology == 1) | (data_lithology == 2))

    data_x_out_keel = data_x[cond_out_keel]
    data_z_out_keel = data_z[cond_out_keel]
    data_lithology_out_keel = data_lithology[cond_out_keel]

    #selecting particles out of the keel, with z deeper than lithosphere + 10 km and lithology = 1 or 2 (lithosphere)
    cond_keel = (data_x >= xcenter - Lcraton/2) & (data_x <= xcenter + Lcraton/2)\
                & (data_z < -(thickness_litho + thickness_sa + thickening + 10.0e3))\
                & ((data_lithology == 1) | (data_lithology == 2))

    data_x_keel = data_x[cond_keel]
    data_z_keel = data_z[cond_keel]
    data_lithology_keel = data_lithology[cond_keel]
    
else: #No mobile belt
    cond_out_keel = ((data_x >= 0) & (data_x < xcenter - Lcraton/2) | (data_x > xcenter + Lcraton/2) & (data_x <= Lx))\
                & (data_z < -(thickness_litho + thickness_sa + 10.0e3))\
                & (data_lithology == 1)

    data_x_out_keel = data_x[cond_out_keel]
    data_z_out_keel = data_z[cond_out_keel]
    data_lithology_out_keel = data_lithology[cond_out_keel]

    #selecting particles out of the keel, with z deeper than lithosphere + 10 km and lithology = 1 (lithosphere)
    cond_keel = (data_x >= xcenter + shift_craton - Lcraton/2) & (data_x <= xcenter + Lcraton/2)\
                & (data_z < -(thickness_litho + thickness_sa + thickening + 10.0e3))\
                & (data_lithology == 1)

    data_x_keel = data_x[cond_keel]
    data_z_keel = data_z[cond_keel]
    data_lithology_keel = data_lithology[cond_keel]

#plotting to test
plt.close()
fig, ax = plt.subplots(1, 1, figsize=(12, 12*(Lz/Lx)), constrained_layout = True)

if(mobile_belt):
    ax.plot(x, interfaces['litho_LAB']/1000, color='black', linestyle='-', linewidth=1.0, alpha=1.0, zorder=30, label='litho interface')
    ax.plot(x, interfaces['litho_HETERO']/1000, color='gray', linestyle='-', linewidth=1.0, alpha=1.0, zorder=30, label='litho interface')
    ax.plot(x, interfaces['air']/1000, color='orange', linestyle='-', linewidth=1.0, alpha=1.0, zorder=30, label='air interface')
else:
    ax.plot(x, interfaces['litho']/1000, color='black', linestyle='-', linewidth=1.0, alpha=1.0, zorder=30, label='litho interface')
    ax.plot(x, interfaces['air']/1000, color='orange', linestyle='-', linewidth=1.0, alpha=1.0, zorder=30, label='air interface')

ax.plot(data_x[data_lithology >= 1][::5]/1000, -data_z[data_lithology >= 1][::5]/1000, "o", color='magenta', markersize=0.2, alpha=1.0, zorder=30, label='lithosphere')
ax.plot(data_x[data_lithology == 0][::5]/1000, -data_z[data_lithology == 0][::5]/1000, "o", color='black', markersize=0.2, alpha=1.0, zorder=30, label='asthenosphere')

ax.plot(data_x_keel/1000, -data_z_keel/1000, "o", color='red', markersize=2.5, alpha=1.0, zorder=30, label='keel')
ax.plot(data_x_out_keel/1000, -data_z_out_keel/1000, "o", color='blue', markersize=2.5, alpha=1.0, zorder=30, label='out keel')

ax.text(0, -10, f"Total eroded particles: {len(data_x_keel) + len(data_x_out_keel)}")
ax.text(3500, -10, f"Time: {np.round(dataset.time[i].values, 2)} Myr")

ax.set_xlim([0, Lx/1000])
ax.set_ylim([Lz/1000 - thickness_sa/1.0e3, 0 - thickness_sa/1.0e3])

ax.set_ylim([Lz/1000, 0])

ax.legend(loc='lower right', fontsize=12, frameon=True, framealpha=1.0, facecolor='white', markerscale=5,
          bbox_to_anchor=(1.2, 0.0))
# legend.get_frame().set_facealpha(1.0)

# ax.set_ylim([100, 0 - thickness_sa/1.0e3])
# ax.set_yticks(np.arange(-40, 100, 20))



# Maping amount of eroded particles over time

In [ ]:
xcenter = (Lx/2) + shift_craton
eroded_particles = []

for i in range(len(dataset.step.values)):
    data_x, data_z, data_ID, data_lithology, data_strain = _read_step(model_path, f"step_{int(dataset.step[i].values*1)}_", ncores)

    if(mobile_belt):
        cond_out_keel = ((data_x >= 0) & (data_x < xcenter - Lcraton/2) | (data_x > xcenter + Lcraton/2) & (data_x <= Lx))\
                    & (data_z < -(thickness_litho + thickness_sa + 10.0e3))\
                    & ( (data_lithology == 1) | (data_lithology == 2))

        data_x_out_keel = data_x[cond_out_keel]
        data_z_out_keel = data_z[cond_out_keel]
        data_lithology_out_keel = data_lithology[cond_out_keel]

        #selecting particles out of the keel, with z deeper than lithosphere + 10 km and lithology = 1 or 2 (lithosphere)
        cond_keel = (data_x >= xcenter - Lcraton/2) & (data_x <= xcenter + Lcraton/2)\
                    & (data_z < -(thickness_litho + thickness_sa + thickening + 10.0e3))\
                    & ((data_lithology == 1) | (data_lithology == 2))

        data_x_keel = data_x[cond_keel]
        data_z_keel = data_z[cond_keel]
        data_lithology_keel = data_lithology[cond_keel]
        
    else: #No mobile belt
        cond_out_keel = ((data_x >= 0) & (data_x < xcenter - Lcraton/2) | (data_x > xcenter + Lcraton/2) & (data_x <= Lx))\
                    & (data_z < -(thickness_litho + thickness_sa + 10.0e3))\
                    & (data_lithology == 1)

        data_x_out_keel = data_x[cond_out_keel]
        data_z_out_keel = data_z[cond_out_keel]
        data_lithology_out_keel = data_lithology[cond_out_keel]

        #selecting particles out of the keel, with z deeper than lithosphere + 10 km and lithology = 1 (lithosphere)
        cond_keel = (data_x >= xcenter + shift_craton - Lcraton/2) & (data_x <= xcenter + Lcraton/2)\
                    & (data_z < -(thickness_litho + thickness_sa + thickening + 10.0e3))\
                    & (data_lithology == 1)

        data_x_keel = data_x[cond_keel]
        data_z_keel = data_z[cond_keel]
        data_lithology_keel = data_lithology[cond_keel]

    eroded_particles.append(len(data_x_keel) + len(data_x_out_keel))
# print(f"Time {dataset.time.values[i]} Myr: Total eroded particles: {len(data_x_keel) + len(data_x_out_keel)}")
    

In [ ]:
save_data = True
# save_data = False

if(save_data):
    np.savetxt(f"{model_path}/{scenario.split('/')[-1]}_eroded_particles_evolution.txt",
               np.asarray([dataset.time.values, eroded_particles]).T,
               delimiter='\t',
               header="Time\tEroded_particles",
               fmt='%2.2f',
               )

In [ ]:
time_er, particles_er = np.loadtxt(f"{model_path}/{scenario.split('/')[-1]}_eroded_particles_evolution.txt", delimiter='\t', skiprows=1, unpack=True)

print(scenario.split('/')[-1])

plt.close()
fig, ax = plt.subplots(1, 1, figsize=(10, 5), constrained_layout = True)

# ax.plot(dataset.time.values, eroded_particles, '-', color='black', alpha=1.0, zorder=30, label='eroded particles')

ax.plot(time_er, particles_er, '-', color='black', alpha=1.0, zorder=30, label='eroded particles')

ax.set_xlim([0, dataset.time.values[-1]])
ax.set_ylim([0, max(eroded_particles)])
# ax.set_ylim([15.0e3, 20.0e3])
ax.set_xlabel('Time (Myr)')
ax.set_ylabel('Eroded particles')
# ax.legend(loc='lower right', fontsize=12, frameon=True, framealpha=1.0, facecolor='white')
ax.grid()

fig.savefig(f"{model_path}/_output/{scenario.split('/')[-1]}_eroded_particles.png", dpi=300)
fig.savefig(f"{model_path}/_output/{scenario.split('/')[-1]}_eroded_particles.pdf", dpi=300)

In [ ]:
!rm {model_path}/step*.txt

# Plotting erosion evolution together

In [ ]:
scenarios = ['/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/keel_stable_DT200_HprodAst_Lx4000km_ck_tbcf',
             '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/weak_keel/keel_stable_DT200_PT1292oC_HprodAst_Lx4000km_ck_tbcf_wk_Cmlit02',
             '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/weak_keel/keel_stable_DT200_PT1292oC_HprodAst_Lx4000km_ck_tbcf_wk_Cmlit03',
             '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/weak_keel/keel_stable_DT200_PT1292oC_HprodAst_Lx4000km_ck_tbcf_wk_Cmlit05',
             '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/shift_craton/keel_stable_DT200_HprodAst_Lx4000km_ck_tbcf_sc',
             '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/mobile_belt/mb_dryol/keel_stable_DT200_HprodAst_Lx4000km_ck_mbdo',
             '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/mobile_belt/mb_wetol/keel_stable_DT200_HprodAst_Lx4000km_ck_mbwo',
             ]

kinds = [r'$C_{m_{\mathrm{lit}}}$ = 1.0 (Ref.)',
         r'$C_{m_{\mathrm{lit}}}$ = 0.2',
         r'$C_{m_{\mathrm{lit}}}$ = 0.3',
         r'$C_{m_{\mathrm{lit}}}$ = 0.5',
         r'Shift craton: 700 km',
         r'Mobile belt: Dry Ol.',
         r'Mobile belt: Wet Ol.',
         ]

color_dict = {'$C_{m_{\mathrm{lit}}}$ = 1.0 (Ref.)':'xkcd:cerulean blue',
              '$C_{m_{\mathrm{lit}}}$ = 0.2':'xkcd:lighter purple',
              '$C_{m_{\mathrm{lit}}}$ = 0.3':'xkcd:bright aqua',
              '$C_{m_{\mathrm{lit}}}$ = 0.5':'xkcd:dirt brown',
              'Shift craton: 700 km':'xkcd:scarlet',
              'Mobile belt: Dry Ol.':'xkcd:dark green',
              'Mobile belt: Wet Ol.':'xkcd:light orange',
              }

# local = True
local = False


fig, ax = plt.subplots(1, 1, figsize=(10, 5), constrained_layout = True)

for scenario, kind in zip(scenarios, kinds):
    name = scenario.split('/')[-1]

    if(local==True):
        model_path = machine_path + scenario
    else:
        external_media = 'Joao_Macedo'
        model_path = f"/media/{machine_path.split('/')[-1]}/{external_media}{scenario}"
    
    fpath = f"{model_path}/{name}_eroded_particles_evolution.txt"
    time, particles = np.loadtxt(fpath, delimiter='\t', unpack=True)
    ax.plot(time, particles, '-', color = color_dict[kind], label=kind)


ax.legend(loc='upper right', ncol=2, fontsize=12, frameon=True, framealpha=1.0, facecolor='white')
ax.set_xlim([0, 1000])
ax.set_ylim([0, 350])
ax.grid()

ax.set_xlabel('Time (Myr)')
ax.set_ylabel('Eroded particles')

In [ ]:
scenarios_sc = ['/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/keel_stable_DT200_HprodAst_Lx4000km_ck_tbcf',
                '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/shift_craton/keel_stable_DT200_HprodAst_Lx4000km_ck_tbcf_sc',]

scenarios_wk = ['/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/keel_stable_DT200_HprodAst_Lx4000km_ck_tbcf',
                '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/weak_keel/keel_stable_DT200_PT1292oC_HprodAst_Lx4000km_ck_tbcf_wk_Cmlit02',
                '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/weak_keel/keel_stable_DT200_PT1292oC_HprodAst_Lx4000km_ck_tbcf_wk_Cmlit03',
                '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/weak_keel/keel_stable_DT200_PT1292oC_HprodAst_Lx4000km_ck_tbcf_wk_Cmlit05',
                ]

scenarios_mb = ['/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/keel_stable_DT200_HprodAst_Lx4000km_ck_tbcf',
                '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/mobile_belt/mb_dryol/keel_stable_DT200_HprodAst_Lx4000km_ck_mbdo',
                '/Doutorado/cenarios/mandyoc/keel/Lx4000km/cold_keel/thermal_bc_fixed/mobile_belt/mb_wetol/keel_stable_DT200_HprodAst_Lx4000km_ck_mbwo',]

kinds_sc = [r'$C_{m_{\mathrm{lit}}}$ = 1.0 (Ref.)',
            r'Shift craton: 700 km',]

kinds_wk = [r'$C_{m_{\mathrm{lit}}}$ = 1.0 (Ref.)',
            r'$C_{m_{\mathrm{lit}}}$ = 0.2',
            r'$C_{m_{\mathrm{lit}}}$ = 0.3',
            r'$C_{m_{\mathrm{lit}}}$ = 0.5',
            ]

kinds_mb = [r'$C_{m_{\mathrm{lit}}}$ = 1.0 (Ref.)',
            r'Mobile belt: Dry Ol.',
            r'Mobile belt: Wet Ol.',
            ]

color_dict = {'$C_{m_{\mathrm{lit}}}$ = 1.0 (Ref.)':'xkcd:cerulean blue',
              '$C_{m_{\mathrm{lit}}}$ = 0.2':'xkcd:lighter purple',
              '$C_{m_{\mathrm{lit}}}$ = 0.3':'xkcd:bright aqua',
              '$C_{m_{\mathrm{lit}}}$ = 0.5':'xkcd:dirt brown',
              'Shift craton: 700 km':'xkcd:scarlet',
              'Mobile belt: Dry Ol.':'xkcd:dark green',
              'Mobile belt: Wet Ol.':'xkcd:light orange',
              }

# local = True
local = False

fig, axs = plt.subplots(3, 1, figsize=(8, 5), constrained_layout = True, sharex=True)

for scenario, kind_sc in zip(scenarios_sc, kinds_sc):
    name = scenario.split('/')[-1]

    if(local==True):
        model_path = machine_path + scenario
    else:
        external_media = 'Joao_Macedo'
        model_path = f"/media/{machine_path.split('/')[-1]}/{external_media}{scenario}"
    
    fpath = f"{model_path}/{name}_eroded_particles_evolution.txt"
    time, particles = np.loadtxt(fpath, delimiter='\t', unpack=True)
    axs[0].plot(time, particles, '-', color = color_dict[kind_sc], label=kind_sc)

for scenario, kind_wk in zip(scenarios_wk, kinds_wk):
    name = scenario.split('/')[-1]

    if(local==True):
        model_path = machine_path + scenario
    else:
        external_media = 'Joao_Macedo'
        model_path = f"/media/{machine_path.split('/')[-1]}/{external_media}{scenario}"
    
    fpath = f"{model_path}/{name}_eroded_particles_evolution.txt"
    time, particles = np.loadtxt(fpath, delimiter='\t', unpack=True)
    axs[1].plot(time, particles, '-', color = color_dict[kind_wk], label=kind_wk)

for scenario, kind_mb in zip(scenarios_mb, kinds_mb):
    name = scenario.split('/')[-1]

    if(local==True):
        model_path = machine_path + scenario
    else:
        external_media = 'Joao_Macedo'
        model_path = f"/media/{machine_path.split('/')[-1]}/{external_media}{scenario}"
    
    fpath = f"{model_path}/{name}_eroded_particles_evolution.txt"
    time, particles = np.loadtxt(fpath, delimiter='\t', unpack=True)
    axs[2].plot(time, particles, '-', color = color_dict[kind_mb], label=kind_mb)

axs[0].set_xlim([0, 1000])

axs[0].set_ylim([0, 60])
axs[1].set_ylim([0, 400])
axs[2].set_ylim([0, 100])

axs[2].set_xlabel('Time (Myr)')

legend_cols = [len(scenarios_sc), len(scenarios_wk), len(scenarios_mb)]
    
for ax, col in zip(axs, legend_cols):
    ax.grid()
    ax.set_ylabel('Eroded particles')
    ax.legend(loc='upper right', ncol=col, fontsize=10, frameon=True)


fig.savefig(f"{machine_path}/opt/scripts/out/plots/comparison_eroded_particles_evolution.png", dpi=300)
fig.savefig(f"{machine_path}/opt/scripts/out/plots/comparison_eroded_particles_evolution.pdf", dpi=300)

# trash

In [ ]:
# mobile_belt = True
mobile_belt = False

# thickness of sticky air layer (m)
thickness_sa = 40 * 1.0e3
# thickness of upper crust (m)
thickness_upper_crust = 20 * 1.0e3
# thickness of lower crust (m)
thickness_lower_crust = 15 * 1.0e3
# total thickness of lithosphere (m)
thickness_litho = 80 * 1.0e3
#shift craton
shift_craton = 0.0e3

if(mobile_belt == False):
        interfaces = {
                "litho": np.ones(Nx) * (thickness_litho + thickness_sa),
                "lower_crust": np.ones(Nx) * (thickness_lower_crust + thickness_upper_crust + thickness_sa),
                "upper_crust": np.ones(Nx) * (thickness_upper_crust + thickness_sa),
                "air": np.ones(Nx) * (thickness_sa),
                }

        dx = Lx/(Nx-1)
        # Lcraton = 600.0e3 #m
        Lcraton = 1200.0e3 #m
        # Lcraton = 2000.0e3 #m
        thickening = thickness_litho + 120.e3 #m
        Ncraton = int(Lcraton//dx) #largura em indices
        
        Nshift = int(shift_craton//dx)
        interfaces['litho'][Nx//2 - Ncraton//2 + Nshift : Nx//2 + Ncraton//2 + Nshift] = thickness_sa + thickening

else: #with mobile belt
    interfaces = {
        "litho_LAB": np.ones(Nx) * (thickness_litho + thickness_sa), #lab horizontal
        "litho_HETERO": np.ones(Nx) * (thickness_litho + thickness_sa), #interface entre central e lateral -  interface 
        "lower_crust": np.ones(Nx) * (thickness_lower_crust + thickness_upper_crust + thickness_sa),
        "upper_crust": np.ones(Nx) * (thickness_upper_crust + thickness_sa),
        "air": np.ones(Nx) * (thickness_sa),
        }

    #Building craton
    dx = Lx/(Nx-1)
    # Lcraton = 600.0e3 #m
    Lcraton = 1200.0e3 #m
#     Lcraton = 2000.0e3 #m
    thickening = thickness_litho + 120.e3 #m

    Ncraton = int(Lcraton//dx) #largura em indices

    Nshift = int(shift_craton//dx)

    interfaces['litho_LAB'][Nx//2 - Ncraton//2 + Nshift : Nx//2 + Ncraton//2 + Nshift] = thickness_sa + thickening
    
    #Building mobile belt
    interfaces['litho_HETERO'][Nx//2 - Ncraton//2 + Nshift : Nx//2 + Ncraton//2 + Nshift] = thickness_sa + thickening
    Lmb = 300.0e3 #length of mobile belt
    N_Lmb = int(Lmb//dx)
    # thinning = 50.0e3
    thinning = 100.0e3

    interfaces['litho_HETERO'][Nx//2 - N_Lmb//2 + Nshift : Nx//2 + N_Lmb//2 + Nshift] = thickness_sa + thickening - thinning
#
    
# #delimiting the limits to extract the particles
# lower_lim = interfaces['litho'] + 10
# upper_lim = interfaces['air'] - 10

In [ ]:
model_path

In [ ]:
ncores = 20
# i = 0
# i = 100
i = 200

data_x, data_z, data_ID, data_lithology, data_strain = _read_step(model_path, f"step_{int(dataset.step[i].values*1)}_", ncores)

xcenter = (Lx/2) + shift_craton

#selecting particles out of the keel, with z deeper than lithosphere km and lithology = 1 (lithosphere)
cond_out_keel = ((data_x >= 0) & (data_x < xcenter - Lcraton/2) | (data_x > xcenter + Lcraton/2) & (data_x <= Lx))\
                & (data_z < -(thickness_litho + thickness_sa + 10.0e3))\
                & (data_lithology == 1)

data_x_out_keel = data_x[cond_out_keel]
data_z_out_keel = data_z[cond_out_keel]
data_lithology_out_keel = data_lithology[cond_out_keel]

#selecting particles out of the keel, with z deeper than lithosphere + 10 km and lithology = 1 (lithosphere)
cond_keel = (data_x >= xcenter - Lcraton/2) & (data_x <= xcenter + Lcraton/2)\
            & (data_z < -(thickness_litho + thickness_sa + thickening + 10.0e3))\
            & (data_lithology == 1)

data_x_keel = data_x[cond_keel]
data_z_keel = data_z[cond_keel]
data_lithology_keel = data_lithology[cond_keel]

#plotting to test
fig, ax = plt.subplots(1, 1, figsize=(12, 12*(Lz/Lx)), constrained_layout = True)

ax.plot(x, interfaces['litho']/1000, color='black', linestyle='-', linewidth=1.0, alpha=1.0, zorder=30, label='litho interface')
ax.plot(x, interfaces['air']/1000, color='orange', linestyle='-', linewidth=1.0, alpha=1.0, zorder=30, label='air interface')

ax.plot(data_x[data_lithology >= 1][::5]/1000, -data_z[data_lithology >= 1][::5]/1000, "o", color='magenta', markersize=0.2, alpha=1.0, zorder=30, label='lithosphere')
ax.plot(data_x[data_lithology == 0][::5]/1000, -data_z[data_lithology == 0][::5]/1000, "o", color='black', markersize=0.2, alpha=1.0, zorder=30, label='asthenosphere')

ax.plot(data_x_keel/1000, -data_z_keel/1000, "o", color='red', markersize=2.5, alpha=1.0, zorder=30, label='keel')
ax.plot(data_x_out_keel/1000, -data_z_out_keel/1000, "o", color='blue', markersize=2.5, alpha=1.0, zorder=30, label='out keel')

ax.text(0, -10, f"Total eroded particles: {len(data_x_keel) + len(data_x_out_keel)}")

ax.set_xlim([0, Lx/1000])
ax.set_ylim([Lz/1000 - thickness_sa/1.0e3, 0 - thickness_sa/1.0e3])

ax.set_ylim([Lz/1000, 0])

ax.legend(loc='lower right', fontsize=12, frameon=True, framealpha=1.0, facecolor='white', markerscale=5,
          bbox_to_anchor=(1.2, 0.0))
# legend.get_frame().set_facealpha(1.0)

# ax.set_ylim([100, 0 - thickness_sa/1.0e3])
# ax.set_yticks(np.arange(-40, 100, 20))



In [ ]:
Lcraton